<br>
<div align="right">Intervenant : Yacine Drideche & Ali Boujaite</div>
<div align="right">Year : 2022</div><br><br><br>
<div align="center"><span style="font-family:Lucida Caligraphy;font-size:32px;color:blue">Data Lead Project </span></div><br><br>
<div align="center"><span style="font-family:Lucida Caligraphy;font-size:24px;color:#e60000">Netflix Movie Recommendation in Real Time</span></div><br><br>
<hr>

# Packages

In [4]:
import pandas as pd
import numpy as np
import math
import re
import os
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import json

from datetime import datetime
import random
import matplotlib
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
import pickle

# Functions

In [1]:
def text_to_df(name):
    '''
    ouvre un fichier texte pour le transformer DataFrame selon le format souhaité
    args:
        name : nom du fichier a lire
    '''
    dict1 = {}
    with open(fr".\data_source\{name}.txt") as fh:

        for line in fh:

            # reads each line and trims of extra the spaces
            # and gives only the valid words
            #command, description = line.strip().split(None, 1)
            line =line.replace('\n','')
            if re.findall('[0-9]+:', line):
                movie_num = line.strip(':')
                dict1[movie_num]=[]

            if not re.findall('[0-9]+:', line):
                #custumer_id, year = line.split(',')
                dict1[movie_num].append(line) 




        #print(dict1)
        array_rating =[]
        array_movie=[]
        array_cusumer_id=[]
        array_year=[]

        for key in dict1:
            for key_array in dict1[key]:
                #print(key_array)
                custumer_id,rating, year = key_array.split(',')
                array_movie.append(key)
                array_cusumer_id.append(custumer_id)
                array_rating.append(rating)
                array_year.append(year)

        d = {'Movie_ID': array_movie, 'User_ID': array_cusumer_id,'Rating':array_rating,'date': array_year }
        df_qualifying = pd.DataFrame(data=d)

    return df_qualifying



# Data

In [6]:
#Read and combine all the sources in the right format :
# combine_data_all = text_to_df('combined_data_1')

#CSV to DataFrame :
df_combine_data = pd.read_csv(".\data_work\combine_data_all.csv")

In [7]:
#Choosing KPI :
df = df_combine_data[['movie_id', 'user_id', 'rating', 'date']]

#Rename the comlumns :
df.columns = ['Movie_ID', 'User_ID', 'Rating', 'date']

#Apply a Datetime format :
df["date"] = pd.to_datetime(df["date"])

c:\Users\Boujaite Ali\Anaconda3\envs\mlflow-env\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


# Basic Statistics

In [8]:
df.head()

,Movie_ID,User_ID,Rating,date
0,1,1488844,3,2005-09-06
1,1,822109,5,2005-05-13
2,1,885013,4,2005-10-19
3,1,30878,4,2005-12-26
4,1,823519,3,2004-05-03


In [18]:
df.describe()["Rating"]

count    1.004805e+08
mean     3.604290e+00
std      1.085219e+00
min      1.000000e+00
25%      3.000000e+00
50%      4.000000e+00
75%      4.000000e+00
max      5.000000e+00
Name: Rating, dtype: float64

In [9]:
df.dtypes #si le type est string changer avec les lignes de code ci-dessous :

# df['Movie_ID'] = df['Movie_ID'].astype(int)
# df['User_ID'] = df['User_ID'].astype(int)
# df['Rating'] = df['Rating'].astype(int)

Movie_ID             int64
User_ID              int64
Rating               int64
date        datetime64[ns]
dtype: object

In [10]:
print("-------------- Number of NaN values --------------\n"+str(df.isnull().sum()))
print("-------------- Total Data --------------")
print("Total number of movie ratings = "+str(df.shape[0]))
print("Number of unique users = "+str(len(np.unique(df["User_ID"]))))
print("Number of unique movies = "+str(len(np.unique(df["Movie_ID"]))))

-------------- Number of NaN values --------------
Movie_ID    0
User_ID     0
Rating      0
date        0
dtype: int64
-------------- Total Data --------------
Total number of movie ratings = 100480507
Number of unique users = 480189
Number of unique movies = 17770


# Creating USER-ITEM sparse matrix from data frame

In [5]:
#startTime = datetime.now()
print("Creating USER_ITEM sparse matrix for train Data")
if os.path.isfile(r".\data_work\TrainUISparseData.npz"):
    print("Sparse Data is already present in your disk, no need to create further. Loading Sparse Matrix")
    TrainUISparseData = sparse.load_npz(r".\data_work\TrainUISparseData.npz")
    print("Shape of Train Sparse matrix = "+str(TrainUISparseData.shape))
    
else:
    print("We are creating sparse data")
    TrainUISparseData = sparse.csr_matrix((df.Rating, (df.User_ID, df.Movie_ID)))
    print("Creation done. Shape of sparse matrix = "+str(TrainUISparseData.shape))
    print("Saving it into disk for furthur usage.")
    sparse.save_npz(r".\data_work\TrainUISparseData.npz", TrainUISparseData)
    print("Done\n")

#print(datetime.now() - startTime)

Creating USER_ITEM sparse matrix for train Data
Sparse Data is already present in your disk, no need to create further. Loading Sparse Matrix
Shape of Train Sparse matrix = (2649430, 17771)


In [12]:
row_index, col_index = TrainUISparseData.nonzero()
rows = np.unique(row_index)

In [11]:
print(TrainUISparseData)

  (6, 30)	3
  (6, 157)	3
  (6, 173)	4
  (6, 175)	5
  (6, 191)	2
  (6, 197)	3
  (6, 241)	3
  (6, 295)	4
  (6, 299)	3
  (6, 329)	4
  (6, 361)	3
  (6, 445)	3
  (6, 457)	5
  (6, 468)	3
  (6, 494)	3
  (6, 501)	3
  (6, 528)	4
  (6, 564)	4
  (6, 580)	3
  (6, 658)	3
  (6, 705)	3
  (6, 706)	3
  (6, 723)	3
  (6, 788)	3
  (6, 825)	3
  :	:
  (2649429, 15844)	5
  (2649429, 15952)	5
  (2649429, 16139)	4
  (2649429, 16242)	4
  (2649429, 16339)	4
  (2649429, 16377)	5
  (2649429, 16390)	5
  (2649429, 16438)	4
  (2649429, 16459)	4
  (2649429, 16668)	5
  (2649429, 16765)	5
  (2649429, 16792)	5
  (2649429, 16879)	3
  (2649429, 16882)	5
  (2649429, 16901)	4
  (2649429, 16922)	5
  (2649429, 17070)	4
  (2649429, 17088)	5
  (2649429, 17157)	3
  (2649429, 17308)	3
  (2649429, 17560)	5
  (2649429, 17580)	4
  (2649429, 17622)	4
  (2649429, 17627)	3
  (2649429, 17692)	2


nous calculons la similarité cosinus entre les n vecteurs non nuls de la sparse matrice :

In [13]:
start = datetime.now()

if not os.path.isfile(r".\data_work\m_m_similarity.npz"):
    print("Movie-Movie Similarity file does not exist in your disk. Creating Movie-Movie Similarity Matrix...")
    
    m_m_similarity = cosine_similarity(TrainUISparseData.T, dense_output = False)
    print("Done")
    print("Dimension of Matrix = {}".format(m_m_similarity.shape))
    print("Storing the Movie Similarity matrix on disk for further usage")
    sparse.save_npz(r".\data_work\m_m_similarity.npz", m_m_similarity)
else:
    print("File exists in the disk. Loading the file...")
    m_m_similarity = sparse.load_npz(r".\data_work\m_m_similarity.npz")
    print("Dimension of Matrix = {}".format(m_m_similarity.shape))
    
print(datetime.now() - start)

Movie-Movie Similarity file does not exist in your disk. Creating Movie-Movie Similarity Matrix...
Done
Dimension of Matrix = (17771, 17771)
Storing the Movie Similarity matrix on disk for further usage
0:22:25.598621


nous cherchons les 3 films les plus similaire selon le **cosinus similarity** et on l'integre un dictionnaire :

In [15]:
movie_ids = np.unique(m_m_similarity.nonzero())

similar_movies_dict = dict()
for movie in movie_ids:
    smlr = np.argsort(-m_m_similarity[movie].toarray().ravel())[1:3]
    similar_movies_dict[movie] = smlr


In [101]:
with open(r".\data_work\similar_movies_dict.pickle", 'wb') as handle:
    pickle.dump(similar_movies_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [102]:
similar_movie_pickle = pd.read_pickle(r".\data_work\similar_movies_dict.pickle")
movie_titles_df = pd.read_csv(r".\data_source\movie_titles.csv",sep = ",", header = None, names=['Movie_ID', 'Year_of_Release', 'Movie_Title'], index_col = "Movie_ID", encoding = "iso8859_2")

In [103]:
similar_movie_pickle

{1: 694,
 2: 11212,
 3: 7293,
 4: 4358,
 5: 5138,
 6: 15747,
 7: 6009,
 8: 14274,
 9: 11070,
 10: 5995,
 11: 7626,
 12: 90,
 13: 10352,
 14: 8252,
 15: 8279,
 16: 5481,
 17: 5730,
 18: 15064,
 19: 6417,
 20: 5726,
 21: 8486,
 22: 9073,
 23: 4495,
 24: 4281,
 25: 11470,
 26: 11910,
 27: 6154,
 28: 8743,
 29: 1460,
 30: 6206,
 31: 10788,
 32: 4266,
 33: 5297,
 34: 13009,
 35: 11806,
 36: 8628,
 37: 10679,
 38: 4093,
 39: 17425,
 40: 1158,
 41: 10332,
 42: 15323,
 43: 6938,
 44: 11789,
 45: 1862,
 46: 17591,
 47: 6710,
 48: 2654,
 49: 8587,
 50: 63,
 51: 15561,
 52: 8524,
 53: 4023,
 54: 921,
 55: 6923,
 56: 5224,
 57: 7773,
 58: 16698,
 59: 11602,
 60: 15761,
 61: 649,
 62: 9499,
 63: 1258,
 64: 5077,
 65: 4312,
 66: 527,
 67: 323,
 68: 14661,
 69: 11008,
 70: 12525,
 71: 3623,
 72: 3694,
 73: 12875,
 74: 14549,
 75: 15102,
 76: 9826,
 77: 10886,
 78: 551,
 79: 9042,
 80: 13751,
 81: 6534,
 82: 9851,
 83: 14963,
 84: 7278,
 85: 2591,
 86: 10323,
 87: 1752,
 88: 14061,
 89: 10531,
 90: 14

In [36]:
movie_titles_df.sample(15)

,Year_of_Release,Movie_Title
Movie_ID,,
684,1995.0,Visitors of the Night
17529,2004.0,Bright Young Things
1994,2005.0,The L Word: Season 2
13900,2000.0,Nightwalker #2: Eternal Darkness
13091,1993.0,Malice
13009,2004.0,Yoga: Step by Step
3990,1991.0,One Good Cop
3578,2005.0,Motley Crue: Carnival of Sins
7949,2004.0,WWE Divas: South of the Border


In [37]:
movieID_GR = 15426

print("Name of the movie -------> "+str(movie_titles_df.loc[movieID_GR][1]))

# print("Number of ratings by users for movie {} is {}".format(movie_titles_df.loc[movieID_GR][1], TrainUISparseData[:,movieID_GR].getnnz()))

# # Meaning of "[:,17765]" means get all the values of column "17765".
# # "getnnz()" give count of explicitly-stored values (nonzeros).

# print("Number of similar movies to {} is {}".format(movie_titles_df.loc[movieID_GR][1], m_m_similarity[movieID_GR].count_nonzero()))

Name of the movie -------> Michael Jackson: Video Greatest Hits: History #2


In [104]:
movie_titles_df.loc[similar_movie_pickle[movieID_GR]]

Year_of_Release                      2004.0
Movie_Title        Michael Jackson: The One
Name: 2419, dtype: object